In [14]:
import urllib.request
from urllib.request import urlopen
from urllib import request as urlreq
import requests
import regex as re
import folium
import json
import copy
import pandas as pd
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup


In [2]:
#0501013B0 = Amoxicillin

url_0501013B0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501013B0&format=json"
response_0501013B0 = urllib.request.urlopen(url_0501013B0)
data_0501013B0 = json.loads(response_0501013B0.read())
flat_data_0501013B0 = pd.json_normalize(data_0501013B0)
flat_data_0501013B0 = flat_data_0501013B0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501013B0 = flat_data_0501013B0.drop(columns=['items', 'quantity'])
flat_data_0501013B0 .rename(columns={'actual_cost': 'Amoxicillin'}, inplace=True)
flat_data_0501013B0

Amoxicillin
row_name         row_id date                   
NHS BARNSLEY CCG 02P    2016-09-01      3019.71
                        2016-10-01      4413.57
                        2016-11-01      5004.58
                        2016-12-01      5929.87
                        2017-01-01      6114.04
...                                         ...
NHS WIRRAL CCG   12F    2021-04-01      4233.48
                        2021-05-01      4873.34
                        2021-06-01      4531.01
                        2021-07-01      4351.03
                        2021-08-01      4266.69

[6378 rows x 1 columns]

In [3]:
#0501030I0 = Doxycycline Hyclate

url_0501030I0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501030I0&format=json"
response_0501030I0 = urllib.request.urlopen(url_0501030I0)
data_0501030I0 = json.loads(response_0501030I0.read())
flat_data_0501030I0 = pd.json_normalize(data_0501030I0)
flat_data_0501030I0 = flat_data_0501030I0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501030I0 = flat_data_0501030I0.drop(columns=['items', 'quantity'])
flat_data_0501030I0.rename(columns={'actual_cost': 'Doxycycline Hyclate'}, inplace=True)
flat_data_0501030I0

Doxycycline Hyclate
row_name         row_id date                           
NHS BARNSLEY CCG 02P    2016-09-01              1997.36
                        2016-10-01              2191.47
                        2016-11-01              2535.35
                        2016-12-01              2739.64
                        2017-01-01              2870.95
...                                                 ...
NHS WIRRAL CCG   12F    2021-04-01              3562.62
                        2021-05-01              3375.06
                        2021-06-01              3674.31
                        2021-07-01              3245.34
                        2021-08-01              3215.68

[6377 rows x 1 columns]

In [4]:
#0501021L0 = Cefalexin

url_0501021L0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501021L0&format=json"
response_0501021L0 = urllib.request.urlopen(url_0501021L0)
data_0501021L0 = json.loads(response_0501021L0.read())
flat_data_0501021L0= pd.json_normalize(data_0501021L0)
flat_data_0501021L0 = flat_data_0501021L0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501021L0 = flat_data_0501021L0.drop(columns=['items', 'quantity'])
flat_data_0501021L0.rename(columns={'actual_cost': 'Cefalexin'}, inplace=True)
flat_data_0501021L0

Cefalexin
row_name         row_id date                 
NHS BARNSLEY CCG 02P    2016-09-01     268.23
                        2016-10-01     283.34
                        2016-11-01     278.30
                        2016-12-01     320.67
                        2017-01-01     295.31
...                                       ...
NHS WIRRAL CCG   12F    2021-04-01    2152.52
                        2021-05-01    2092.57
                        2021-06-01    2263.08
                        2021-07-01    2028.57
                        2021-08-01    2176.44

[6372 rows x 1 columns]

In [5]:
join_1 = flat_data_0501013B0.join(flat_data_0501030I0, lsuffix='row_id', rsuffix='row_id')
all_antibiotics_merged = join_1.join(flat_data_0501021L0, lsuffix='row_id', rsuffix='row_id')
all_antibiotics_merged.fillna(0, inplace=True)
all_antibiotics_merged['Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)']= all_antibiotics_merged.iloc[:, -3:].sum(axis=1)
all_antibiotics = all_antibiotics_merged.reset_index()
all_antibiotics.rename(columns={'row_name': 'Clinical Commissioning Group (CCG)', 'row_id': 'CCG code', 'date': 'Date'}, inplace=True)
all_antibiotics

,Clinical Commissioning Group (CCG),CCG code,Date,Amoxicillin,Doxycycline Hyclate,Cefalexin,"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)"
0,NHS BARNSLEY CCG,02P,2016-09-01,3019.71,1997.36,268.23,5285.30
1,NHS BARNSLEY CCG,02P,2016-10-01,4413.57,2191.47,283.34,6888.38
2,NHS BARNSLEY CCG,02P,2016-11-01,5004.58,2535.35,278.30,7818.23
3,NHS BARNSLEY CCG,02P,2016-12-01,5929.87,2739.64,320.67,8990.18
4,NHS BARNSLEY CCG,02P,2017-01-01,6114.04,2870.95,295.31,9280.30
...,...,...,...,...,...,...,...
6373,NHS WIRRAL CCG,12F,2021-04-01,4233.48,3562.62,2152.52,9948.62
6374,NHS WIRRAL CCG,12F,2021-05-01,4873.34,3375.06,2092.57,10340.97
6375,NHS WIRRAL CCG,12F,2021-06-01,4531.01,3674.31,2263.08,10468.40
6376,NHS WIRRAL CCG,12F,2021-07-01,4351.03,3245.34,2028.57,9624.94


In [6]:
all_antibiotics_plot = all_antibiotics.groupby(['Date']).sum()
all_antibiotics_plot = all_antibiotics_plot.reset_index()
all_antibiotics_plot = all_antibiotics_plot.round(2)
all_antibiotics_plot

,Date,Amoxicillin,Doxycycline Hyclate,Cefalexin,"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)"
0,2016-09-01,674547.06,299548.12,104026.20,1078121.38
1,2016-10-01,915656.47,334579.69,101124.46,1351360.62
2,2016-11-01,1086092.74,371928.48,104744.19,1562765.41
3,2016-12-01,1332072.84,420979.41,109233.93,1862286.18
4,2017-01-01,1260825.44,430970.81,104245.89,1796042.14
5,2017-02-01,999002.31,362181.47,94747.56,1455931.34
6,2017-03-01,951099.41,381835.65,106471.54,1439406.60
7,2017-04-01,767132.16,312668.72,87970.82,1167771.70
8,2017-05-01,795452.59,334777.04,95411.40,1225641.03
9,2017-06-01,649513.68,312937.15,95998.03,1058448.86


In [7]:
fig_1_data = all_antibiotics_plot.copy()
fig_1_data.rename(columns={
"Amoxicillin": "Cost (£) of Amozicillin",
"Doxycycline Hyclate": "Cost (£) of Doxycycline Hyclate", 
"Cefalexin": "Cost (£) of Cefalexin",
"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)": "Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin"}, 
inplace=True)
fig_1_data.index.name = 'Unique ID'

In [8]:
pd.options.plotting.backend = "plotly"
fig = px.bar(all_antibiotics_plot, x='Date', y= ["Amoxicillin", "Doxycycline Hyclate", 'Cefalexin'],
color_discrete_sequence=["#003087", "#0072CE", "#41B6E6"],
labels={"value": "Cost (£)", "variable": "Antibiotic"})
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    font = dict(family = "Arial", size = 16),
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
)
fig.show()

In [9]:
#CCG pop from NHS digital
month_year_variable = datetime.now().strftime('%B-%Y').lower()
url = "https://digital.nhs.uk/data-and-information/publications/statistical/patients-registered-at-a-gp-practice/%s" %month_year_variable 
response = urllib.request.urlopen(url)
soup = BeautifulSoup(response.read(), "lxml")
data = soup.select_one("a[href*='gp-reg-pat-prac-all.csv']")
if data != None: 
    csv_url = data['href']
    req = requests.get(csv_url)
    url_content = req.content
    csv_file = open('assets/data/ccg_pop.csv', 'wb')
    csv_file.write(url_content)
    csv_file.close()
else:
    last_month = datetime.now() - relativedelta(months=1)
    last_month_year_variable = last_month.strftime('%B-%Y').lower()
    url = "https://digital.nhs.uk/data-and-information/publications/statistical/patients-registered-at-a-gp-practice/%s" %last_month_year_variable
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response.read(), "lxml")
    data = soup.select_one("a[href*='gp-reg-pat-prac-all.csv']")
    csv_url = data['href']
    req = requests.get(csv_url)
    url_content = req.content
    csv_file = open('assets/data/ccg_pop.csv', 'wb')
    csv_file.write(url_content)
    csv_file.close()
df1 = pd.read_csv('assets/data/ccg_pop.csv')
CCG_pop = df1.groupby(['CCG_CODE']).sum().reset_index()
CCG_pop.rename(columns={'CCG_CODE': 'CCG code', 'NUMBER_OF_PATIENTS': 'Number of patients registered at GP practices'}, inplace=True) 

In [10]:
current_year = datetime.now().year
current_year_str = str(current_year)

In [39]:
all_antibiotics["Date"] = pd.to_datetime(all_antibiotics["Date"]).apply(lambda x: x.strftime("%Y"))
all_antibiotics_current_year = all_antibiotics.loc[all_antibiotics['Date'] == current_year_str]
df1 = all_antibiotics_current_year.groupby(["CCG code", "Clinical Commissioning Group (CCG)"]).sum()
df2 = df1.drop(columns=['Amoxicillin', 'Doxycycline Hyclate', 'Cefalexin'])
df3 = df2.reset_index()
df4 = df3.join(CCG_pop, rsuffix='CCG code')
df5 = df4.drop(columns=['CCG codeCCG code'])
df5.rename(columns = {"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin (£)": "Cost (£) of Amoxicillin, Doxycycline Hyclate,and Cefalexin in %s" %current_year_str}, inplace=True)
df5["Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients in %s" %current_year_str] = df5["Cost (£) of Amoxicillin, Doxycycline Hyclate,and Cefalexin in %s" %current_year_str]/(df5["Number of patients registered at GP practices"]/1000)
df6 = df5.reset_index(drop = True)
df7 = df6.rename(columns = {'CCG code': 'ODS CCG code'})
df7 = df7.round(2)
df7.index.name = 'Unique ID'
df7

,ODS CCG code,Clinical Commissioning Group (CCG),"Cost (£) of Amoxicillin, Doxycycline Hyclate,and Cefalexin in 2021",Number of patients registered at GP practices,"Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients in 2021"
Unique ID,,,,,
0,00L,NHS NORTHUMBERLAND CCG,67090.31,332846,201.57
1,00N,NHS SOUTH TYNESIDE CCG,33526.55,158679,211.29
2,00P,NHS SUNDERLAND CCG,58003.70,285248,203.34
3,00Q,NHS BLACKBURN WITH DARWEN CCG,35622.33,180717,197.12
4,00R,NHS BLACKPOOL CCG,41072.64,175714,233.75
...,...,...,...,...,...
101,D9Y0V,"NHS HAMPSHIRE, SOUTHAMPTON AND ISLE OF WIGHT CCG",280181.36,1683293,166.45
102,M1J4Y,"NHS BEDFORDSHIRE, LUTON AND MILTON KEYNES CCG",156732.29,1067274,146.85
103,M2L0M,"NHS SHROPSHIRE, TELFORD AND WREKIN CCG",103865.97,517023,200.89


In [72]:
def ons_geoportal_file_download(search_url, url_start, string_filter):
  url_2 = '/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
  response = urlreq.urlopen(search_url)
  soup = BeautifulSoup(response.read(), "lxml")
  data_url = soup.find_all('a', href=re.compile(string_filter))[-1].get('href')
  full_url = url_start + data_url + url_2
  with urlopen(full_url) as response:
      json_file = json.load(response)
  return json_file

search_url = "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/"
url_start = "https://services1.arcgis.com"
string_filter = "CCG_APR"
ccg_code_map_json = ons_geoportal_file_download(search_url, url_start, string_filter)
ccg_code_map_df = pd.json_normalize(ccg_code_map_json['features'])
column_ods_code = ccg_code_map_json['fields'][1]['name'].lower()
column_ons_code_1 = ccg_code_map_json['fields'][0]['name'].lower()
ccg_code_map_df = ccg_code_map_df.iloc[:,:2]
ccg_code_map_df.columns = ccg_code_map_df.columns.str.lower()
ccg_code_map_df.rename(columns={'attributes.%s' %column_ons_code_1 :'ONS CCG code', 'attributes.%s' %column_ods_code: 'ODS CCG code'}, inplace=True)

In [73]:
final_df = ccg_code_map_df.merge(df7, how='outer', on= 'ODS CCG code')
final_df

,ONS CCG code,ODS CCG code,Clinical Commissioning Group (CCG),"Cost (£) of Amoxicillin, Doxycycline Hyclate,and Cefalexin in 2021",Number of patients registered at GP practices,"Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients in 2021"
0,E38000006,02P,NHS BARNSLEY CCG,52380.33,265011,197.65
1,E38000007,99E,NHS BASILDON AND BRENTWOOD CCG,57715.83,285098,202.44
2,E38000008,02Q,NHS BASSETLAW CCG,19454.81,121413,160.24
3,E38000014,00Q,NHS BLACKBURN WITH DARWEN CCG,35622.33,180717,197.12
4,E38000015,00R,NHS BLACKPOOL CCG,41072.64,175714,233.75
...,...,...,...,...,...,...
101,E38000253,D9Y0V,"NHS HAMPSHIRE, SOUTHAMPTON AND ISLE OF WIGHT CCG",280181.36,1683293,166.45
102,E38000254,X2C4Y,NHS KIRKLEES CCG,93539.43,448906,208.37
103,E38000255,A3A8R,NHS NORTH EAST LONDON CCG,330105.36,2333359,141.47
104,E38000256,W2U3Z,NHS NORTH WEST LONDON CCG,336288.58,2731518,123.11


In [75]:
search_url = "https://ons-inspire.esriuk.com/arcgis/rest/services/Health_Boundaries/"
url_start = "https://ons-inspire.esriuk.com"
string_filter = "Clinical_Commissioning_Groups_April"
data_ccg_geojson = ons_geoportal_file_download(search_url, url_start, string_filter)

In [78]:
github_url = 'https://raw.githubusercontent.com/nhs-pycom/coding_club_nhs_data_ingestion/main/ccg_shapefile.geojson'
response = urlopen(github_url)
data_ccg_geojson_test = json.loads(response.read())

In [83]:
data_ccg_geojson

{'displayFieldName': 'CCG21CD',
 'fieldAliases': {'objectid': 'FID',
  'ccg21cd': 'ccg21cd',
  'ccg21nm': 'ccg21nm',
  'bng_e': 'bng_e',
  'bng_n': 'bng_n',
  'long': 'long',
  'lat': 'lat',
  'st_area(shape)': 'st_area(shape)',
  'st_length(shape)': 'st_length(shape)'},
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
 'fields': [{'name': 'objectid', 'type': 'esriFieldTypeOID', 'alias': 'FID'},
  {'name': 'ccg21cd',
   'type': 'esriFieldTypeString',
   'alias': 'ccg21cd',
   'length': 9},
  {'name': 'ccg21nm',
   'type': 'esriFieldTypeString',
   'alias': 'ccg21nm',
   'length': 59},
  {'name': 'bng_e', 'type': 'esriFieldTypeInteger', 'alias': 'bng_e'},
  {'name': 'bng_n', 'type': 'esriFieldTypeInteger', 'alias': 'bng_n'},
  {'name': 'long', 'type': 'esriFieldTypeDouble', 'alias': 'long'},
  {'name': 'lat', 'type': 'esriFieldTypeDouble', 'alias': 'lat'},
  {'name': 'st_area(shape)',
   'type': 'esriFieldTypeDouble',
   'alias': 'st_area(

In [86]:
# with urlopen('https://openprescribing.net/api/1.0/org_location/?org_type=ccg') as response:
#      data_ccg_geojson = json.load(response)

tooltip_text = { x: y for x, y in zip(final_df['ONS CCG code'], final_df['Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients in %s' %current_year_str])}
tooltip_text_2 = { x: y for x, y in zip(final_df['ONS CCG code'], final_df['Number of patients registered at GP practices'].apply(str))}

for idx,x in enumerate(data_ccg_geojson['features']):
    this_tooltip_text = tooltip_text[x['attributes']['ccg21cd']]
    data_ccg_geojson['features'][idx]['attributes']['Cost (£) per 1000 GP registered population'] = this_tooltip_text

for idx,x in enumerate(data_ccg_geojson['features']):
    this_tooltip_text_2 = tooltip_text_2[x['attributes']['ccg21cd']]
    data_ccg_geojson['features'][idx]['attributes']['GP registered population'] = this_tooltip_text_2

In [88]:
def check_to_include(feature):
    return (feature['geometry'] is not None)

# def transform(feature):
#     new_feature = copy.deepcopy(feature)
#     y = new_feature['attributes']
#     del y['ons_code']
#     return feature

data_ccg_geojson_2 = data_ccg_geojson.copy()
data_ccg_geojson_2['features'] = [x for x in data_ccg_geojson['features'] if check_to_include(x)]
data_ccg_geojson_2

{'displayFieldName': 'CCG21CD',
 'fieldAliases': {'objectid': 'FID',
  'ccg21cd': 'ccg21cd',
  'ccg21nm': 'ccg21nm',
  'bng_e': 'bng_e',
  'bng_n': 'bng_n',
  'long': 'long',
  'lat': 'lat',
  'st_area(shape)': 'st_area(shape)',
  'st_length(shape)': 'st_length(shape)'},
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
 'fields': [{'name': 'objectid', 'type': 'esriFieldTypeOID', 'alias': 'FID'},
  {'name': 'ccg21cd',
   'type': 'esriFieldTypeString',
   'alias': 'ccg21cd',
   'length': 9},
  {'name': 'ccg21nm',
   'type': 'esriFieldTypeString',
   'alias': 'ccg21nm',
   'length': 59},
  {'name': 'bng_e', 'type': 'esriFieldTypeInteger', 'alias': 'bng_e'},
  {'name': 'bng_n', 'type': 'esriFieldTypeInteger', 'alias': 'bng_n'},
  {'name': 'long', 'type': 'esriFieldTypeDouble', 'alias': 'long'},
  {'name': 'lat', 'type': 'esriFieldTypeDouble', 'alias': 'lat'},
  {'name': 'st_area(shape)',
   'type': 'esriFieldTypeDouble',
   'alias': 'st_area(

In [69]:
frame = folium.Figure(width=700, height=500)
fig_2 = folium.Map(
    location=[53, 1],
    tiles="cartodbpositron",
    zoom_start=6).add_to(frame)
folium.Choropleth(
    geo_data = data_ccg_geojson_test,
    name="choropleth",
    data= final_df,
    columns=["ONS CCG code", "Cost (£) of Amoxicillin, Doxycycline Hyclate, and Cefalexin per 1000 GP registered patients in %s" %current_year_str],
    key_on="feature.properties.CCG21CD",
    fill_color= "BuPu",
    fill_opacity=1,
    line_opacity=0.5,
    legend_name="Prescribing cost (£) per 1000 GP registered population in %s" %current_year_str,
    highlight = True
).add_to(fig_2)
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.5, 
                                'weight': 0.1}
data_on_hover = folium.features.GeoJson(data = data_ccg_geojson_2, style_function=style_function, control=False, highlight_function=highlight_function, tooltip=folium.features.GeoJsonTooltip(
    fields=['name', 'code', 'GP registered population', 'Cost (£) per 1000 GP registered population'],
    aliases=['CCG name: ', 'CCG code: ', 'GP registered population: ', 'Cost (£) per 1000 GP registered population: '],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")))
fig_2.add_child(data_on_hover)
fig_2.keep_in_front(data_on_hover)
folium.LayerControl().add_to(fig_2)
fig_2


KeyError: 'type'

In [ ]:
fig_2.save("assets/folium/folium_obj.html", "w")